# 01. Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os

# 02. Importing the data

In [2]:
# Creating a path 
path= r'C:\Users\amore\Documents\Data Analyst course\Data Immersion\Achievement 4_python\Instacard_basket_analysis_29_01_2023'

In [3]:
# Importing the customers dataframe
df_customers=pd.read_csv(os.path.join(path,'02 Data','Original Data','customers.csv'))

In [4]:
# Importing the orders_products dataframe
df_ords_prods = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data','orders_products_merged_update_2_clean.pkl'))

# 03. Task 4.9 - Part 1 

**Q4. Wrangle the data so that it follows consisent logic; for example, rename columns with illogical names and drop columns that don't add anything to your analysis**

In [5]:
#checking the new data 
df_customers.shape

(206209, 10)

In [6]:
#checking the new data
df_customers.tail()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
206204,168073,Lisa,Case,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Jeremy,Robbins,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Doris,Richmond,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Rose,Rollins,Female,California,27,4/1/2020,1,married,99799
206208,80148,Cynthia,Noble,Female,New York,55,4/1/2020,1,married,57095


In [7]:
#Checking the data types
df_customers.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [8]:
# Descriptive statistics of the new dataframe
df_customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


There isn't anything looking strange in the descriptive statistics, all values appear logical

In [9]:
# Renaming columns 
df_customers.rename(columns= {'Gender' : 'gender','First Name': 'name', 'Surnam' : 'surname','STATE': 'state',
                             'Age': 'age'}, inplace = True)

In [10]:
# Checking the changes
df_customers.head()

,user_id,name,surname,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


**Q5 Complete the fundamental data quality and consistency checks you've learned throughout this Achievement; for example, check for and address missing values and duplicates and convert any mixed type data**

In [11]:
#Checking for null values 
df_customers.isnull().sum()

user_id             0
name            11259
surname             0
gender              0
state               0
age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

There is a lot of missing values in the Name column. Given the nature of the column it is not possible to impute any value, thus I think that the best option is to drop that column and the "surname" column, as it doesn't make much sense to keep just surnames without names


In [12]:
# droping the name and surname columns 
df_customers = df_customers.drop(columns =['name', 'surname'])

In [13]:
#checking the results
df_customers.head()

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [14]:
# Changing the data type of the user_id column to string
df_customers['user_id'] = df_customers['user_id'].astype ('str')

In [15]:
# Changing the data type of the income column to float
df_customers['income'] = df_customers['income'].astype('float')

In [16]:
# Changing the data type of the date_joined column to datetime64
df_customers['date_joined'] = df_customers['date_joined'].astype('datetime64')

In [17]:
#Checking the changes in data types
df_customers.dtypes

user_id                 object
gender                  object
state                   object
age                      int64
date_joined     datetime64[ns]
n_dependants             int64
fam_status              object
income                 float64
dtype: object

In [18]:
# Checking for mixed-type data
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col, ' mixed')
  else: print(col, ' consistent')

user_id  consistent
gender  consistent
state  consistent
age  consistent
date_joined  consistent
n_dependants  consistent
fam_status  consistent
income  consistent


There is no mixed data

In [19]:
# checking for duplicates
df_dups = df_customers[df_customers.duplicated()]

In [20]:
df_dups

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income


There aren't any duplicates! 

**Q6. Combine your customer data with the rest of your prepared Instacart data**

In [21]:
#Checking that the user_id column of both dataframes, which will be used to combine the dataframes is the same data type
df_customers['user_id'].dtype

dtype('O')

In [22]:
df_ords_prods['user_id'].dtype

dtype('int64')

In [23]:
#changing the data type of the user_id column in the df_ords_prods dataframe
df_ords_prods['user_id'] = df_ords_prods['user_id'].astype ('str')

In [24]:
# Checking the change
df_ords_prods['user_id'].dtype

dtype('O')

In [25]:
# Merging the dataframes using the user_id column
df_def = df_ords_prods.merge(df_customers, on = 'user_id')

In [26]:
#Checking the new dataframe 
df_def.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,orders_day_of_week,...,spending_flag,order_frequency,frequency_flag,gender,state,age,date_joined,n_dependants,fam_status,income
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,Low Spender,8.0,Frequent customer,Male,Minnesota,81,2019-08-01,1,married,49620.0
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,Low Spender,8.0,Frequent customer,Male,Minnesota,81,2019-08-01,1,married,49620.0
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,prior,1,5,...,Low Spender,8.0,Frequent customer,Male,Minnesota,81,2019-08-01,1,married,49620.0
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,prior,10,5,...,Low Spender,8.0,Frequent customer,Male,Minnesota,81,2019-08-01,1,married,49620.0
4,1000,Apricots,18,10,12.9,505689,138,prior,9,6,...,Low Spender,8.0,Frequent customer,Male,Minnesota,81,2019-08-01,1,married,49620.0


In [27]:
df_def.shape

(32404859, 33)

**Q8 Exporting the new dataframe as a pickle file** 

In [28]:
df_def.to_pickle(os.path.join(path, '02 Data','Prepared Data','orders_products_merged_update3.pkl'))